In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src import read_nz_file, read_jg_file, update_meta_data, split_df, aggregate_files, add_moving_window
from sklearn.model_selection import train_test_split

processed data - intermediate step

> raw data

> clean data

> preprocess data : store it in DB (better compared to file format) 

    - data lake
    
    or
    
    - DB model based SQL (Nice to have but not required if we deceide to save the data as a Feather file)

> 

## 1. Load and clean raw data

files from SensorLog iOS app has in total over 70 colums and precision of 12 decimal figures. The output file is over 135 MB, which is too large for GitHub. GitHub restricts the file size, therefore these files from SensorLog must be cleaned.

Below summary of steps which is done only for iOS files:

- read raw data as csv files
- remove unnecessary columns (captured in list 'remove_cols' below)
- round to 6 decimal places to reduce the size of files
- output dataframe as csv
- upload the csv on GitHub

# 2. Meta data

The data on different data files is captured in meta dataframe below:
- file name
- user (nz or jg)
- activity (running/cycling/walking/sitting)
- pocket (in which pocket handy was during the activity)
- position_x
- position_y


# 3. View data

Sensor activity data is captured from 2 different Apps:
- SensorLog (iOS) by user 'nz'
- AndrioSensor (Andriod) by user 'jg'

**Response**: 'Acivity' with 4 classes: running/walking/cycling/sitting



In [ ]:
update_meta_data()

In [ ]:
meta = pd.read_csv('data/meta.csv')
meta

In [ ]:
train = pd.DataFrame()
test = pd.DataFrame()


for file, user, activity in zip(meta['file'], meta['user'], meta['activity']):
    if user == 'nz':
        df = read_nz_file(file, activity)

    elif user == 'jg':
        df = read_jg_file(file, activity)

    print(file, user, activity, df.shape)

    # create synthetic features

    # split into train-test
    hz = 20
    my_train_files, my_test_files = split_df(df, hz = hz, test_proportion = 0.2)

    print(f'Train: {[len(i) for i in my_train_files]}')
    print(f'Test: {[len(i) for i in my_test_files]}')

    # aggregate data points (try moving average) transform to mean, sd, ...
    for i, (v_train, v_test) in enumerate(zip(my_train_files, my_test_files)):
        # i.reset_index(drop = True)
        v_train = add_moving_window(v_train, hz_old_data = hz, seconds = 20, step_size = 20)
        my_train_files[i] = v_train

        v_test = add_moving_window(v_test, hz_old_data = hz, seconds = 20, step_size = 20)
        my_test_files[i] = v_test

    print(f'Train: {[len(i) for i in my_train_files]}')
    print(f'Test: {[len(i) for i in my_test_files]}')

    # append to train and test
    train = aggregate_files(my_train_files, train)
    test = aggregate_files(my_test_files, test)

# X - y split for train and test data, shuffle data!?

# TODO: Fix train test split to give us 80 / 20 ratio

In [ ]:
train